In [ ]:
!pip install altair vega_datasets

In [ ]:
import pandas as pd


# Destilacion de ultima capa

In [ ]:
folder="./Cifar10/ResNet101/exp1/students"
data = pd.read_csv(folder+"/summary.csv") 


def plot_KD(phase="test"):
    detalle=['test_acc', 'test_teacher/acc', 'test_loss', 'test_eval',
           'train_acc', 'train_teacher/acc', 'train_loss', 'train_eval', 'epoch', 'distillation', 'temp']
    bar=alt.Chart(data).mark_point().encode(
        alt.X('temp', scale=alt.Scale(zero=False,base=10,type='log', )),
        alt.Y('%s_acc'%phase, scale=alt.Scale(zero=False, padding=1,)),
        shape='distillation',
        color='student',
        tooltip=detalle
    ).interactive()

    rule = alt.Chart(data).mark_rule(color='blue').encode(
        y='mean(%s_teacher/acc):Q'%phase
    )
    rule = alt.Chart(data).mark_rule(color='red').encode(
        y='mean(%s_teacher/acc):Q'%phase
    )



    return(bar + rule).properties(width=600,height=400)
plot_KD()

## Destilacion con features usando KD

In [6]:
!ls Cifar10/ResNet101/students

MobileNet-KD  MobileNet-KDCE  ResNet18-KD  ResNet18-KDCE  summary.csv


In [7]:
import altair as alt

folder="./Cifar10/ResNet101/exp4/students/"
mid_folders=[""]#"MobileNet-KD       MobileNet-KDCE  ResNet18-KD     ResNet18-KDCE".split()
ce_test_acc={'MobileNet':87.8,
             'ResNet18':90.58}
ce_train_acc={'MobileNet':95.73,
             'ResNet18':98.15}
source = pd.concat([pd.read_csv(folder+mid_folder+"/summary.csv") for mid_folder in mid_folders])


source=source.fillna(1)

source['layer']=[row['layer']-2 if row['student']=="ResNet18" else row['layer'] for i,row in source.iterrows()]
source['last_layer']=['KD' if row['last_layer']==1 else row['last_layer'] for i,row in source.iterrows()]
source['last_layer']=['KD-CE' if row['last_layer']=='KD_CE' else row['last_layer'] for i,row in source.iterrows()]
source['last_layer']=['KD-CE' if row['last_layer']=='KDCE,T-8' else row['last_layer'] for i,row in source.iterrows()]
source['last_layer']=['KD' if row['last_layer']=='KD,T-8' else row['last_layer'] for i,row in source.iterrows()]
source.to_csv('todo.csv')

source['student last_layer']=[row['student']+","+row['last_layer'] for i,row in source.iterrows()]
source['student_CE_train_Acc']=[ce_test_acc[row['student']] for i,row in source.iterrows()]
source['student_CE_test_Acc']=[ce_test_acc[row['student']] for i,row in source.iterrows()]


SyntaxError: invalid syntax (<ipython-input-7-e8438b603893>, line 4)

In [ ]:

def plot_acc(phase="test"):

    
    detalle=["student","teacher","distillation","layer",'last_layer','test_acc','train_acc',
             'train_loss','epoch',"student_CE_%s_Acc"%phase]#,"lambda"]
    
    
    bar=alt.Chart(source).mark_point().encode(
        alt.X('layer:O', scale=alt.Scale(zero=False,padding=1)),
        alt.Y('%s_acc'%phase, scale=alt.Scale(zero=False, padding=1)),
        color=alt.Color('student last_layer'),
        shape='distillation',
        tooltip=detalle
    ).interactive()


    rule1 = alt.Chart(source).mark_rule(color='blue').encode(
        y='mean(%s_teacher/acc):Q'%phase
    )
    rule2 = alt.Chart(source).mark_rule(color='red').encode(
        y='mean(student_CE_%s_Acc):Q'%phase
    )

    return (bar + rule1 + rule2).properties(width=600,height=400)
    
plot_acc()

In [ ]:
plot_acc("train")

In [ ]:

def plot_acc(phase="test",filter=None):

    
    detalle=["student","teacher","distillation","layer",'last_layer','test_acc','train_acc',
             'train_loss','epoch',"student_CE_%s_Acc"%phase]#,"lambda"]
    
    if filter is not None:
        S= source[source[filter[0]].isin(filter[1])]
    else:
        S=source
        
    bar=alt.Chart(S).mark_point().encode(
        alt.X('layer:O', scale=alt.Scale(zero=False,padding=1)),
        alt.Y('%s_acc'%phase, scale=alt.Scale(zero=False, padding=1)),
        color=alt.Color('student last_layer'),
        shape='distillation',
        tooltip=detalle
    ).interactive()


    rule1 = alt.Chart(source).mark_rule(color='blue').encode(
        y='mean(%s_teacher/acc):Q'%phase
    )
    rule2 = alt.Chart(source).mark_rule(color='red').encode(
        y='mean(student_CE_%s_Acc):Q'%phase
    )

    return (bar + rule1 + rule2).properties(width=600,height=400)
    
plot_acc(filter=('student',['ResNet18']))

In [ ]:
def plot_gen(field="eval",phase="test"):

    
    detalle=["student","teacher","distillation","layer","lambda"]

    bar=alt.Chart(source).mark_point().encode(
        alt.X('layer', scale=alt.Scale(zero=False,padding=1)),
        alt.Y('%s_%s'%(phase,field), scale=alt.Scale(zero=False, padding=1)),
         color=alt.Color('lambda:Q', bin=alt.Bin(maxbins=3),  scale=alt.Scale(scheme="plasma" )),
        shape='distillation',
        tooltip=detalle
    ).interactive()


    return bar

In [ ]:
plot_gen()

In [ ]:
source.head()